In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
data_dir =  '/content/drive/MyDrive/Trabalho Topicos IA1/Dados/novos_inmet/'
files = [
         'generatedBy_react-csv.csv',
         'generatedBy_react-csv(1).csv',
         'generatedBy_react-csv(2).csv',
         'generatedBy_react-csv(3).csv',
         'generatedBy_react-csv(4).csv',
         'generatedBy_react-csv(5).csv'
         ]
separator = ';'
skiprows=0

In [17]:
def merge_csvs_from_list(files):
    dataframes = []
    for file in files:
        df_aux_append = pd.read_csv(data_dir+file, skiprows=skiprows, sep=separator)
        dataframes.append(df_aux_append)

    df_aux = pd.concat(dataframes, ignore_index=True)

    return df_aux

def substituir_valores(df, columns):
    df_aux = df.copy()

    df_aux[float_columns] = df_aux[float_columns].replace(',', '.', regex=True).astype(float)
    df_aux.replace(-9999, np.nan, inplace=True)

    return df_aux

def processar_coluna_data(df):
    df_aux = df.copy()

    df_aux['Data'] = pd.to_datetime(df_aux['Data'], format='%d/%m/%Y', errors='coerce')
    df_aux['Data'] = df_aux['Data'].dt.strftime('%Y-%m-%d')
    df_aux.rename(columns={'Data': 'DATA'}, inplace=True)

    return df_aux

def padronizar_hora(hora):
    hora = str(hora)
    if ':' in hora: return hora
    else:
      hora = hora.zfill(4)

      hora = f"{hora[:2]}:{hora[2:]}"

      return hora

def processar_coluna_hora(df):
    df_aux = df.copy()

    df_aux['Hora (UTC)'] = df_aux['Hora (UTC)'].apply(padronizar_hora)
    df_aux.rename(columns={'Hora (UTC)': 'HORA'}, inplace=True)

    return df_aux

def interpolate_slinear(df, columns):
    df_aux = df.copy()

    for col in columns:
        df_aux[col] = df_aux[col].interpolate(method='slinear')

    return df_aux

def trocar_nan_para_0(df, columns):
    df_aux = df.copy()

    for col in columns:
        df_aux[col] = df_aux[col].fillna(0)

    return df_aux

In [7]:
files_to_convert = files[0:6]
files_to_convert

['generatedBy_react-csv.csv',
 'generatedBy_react-csv(1).csv',
 'generatedBy_react-csv(2).csv',
 'generatedBy_react-csv(3).csv',
 'generatedBy_react-csv(4).csv',
 'generatedBy_react-csv(5).csv']

In [5]:
df2 = pd.read_csv('/content/drive/MyDrive/Trabalho Topicos IA1/final_inmet_data.csv', sep=';')

In [18]:
df2['HORA'] = df2['HORA'].apply(padronizar_hora)
df2

,Unnamed: 0,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,0,2007-05-18,00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2007-05-18,01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2007-05-18,02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2007-05-18,03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2007-05-18,04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147139,147139,2024-02-28,05:00,25.7,26.3,25.6,99.0,99.0,97.0,25.5,26.0,25.3,1005.5,1006.6,1005.2,3.2,66.0,8.0,0.0,0.0
147140,147140,2024-02-28,06:00,26.7,26.8,25.2,97.0,100.0,97.0,26.1,26.2,25.0,1005.1,1005.8,1005.0,1.7,352.0,6.3,0.0,0.0
147141,147141,2024-02-28,07:00,26.9,26.9,26.3,95.0,97.0,95.0,26.0,26.1,25.7,1004.5,1005.2,1004.5,2.1,15.0,4.9,0.0,0.0
147142,147142,2024-02-28,08:00,26.8,27.0,26.7,93.0,95.0,93.0,25.6,26.0,25.6,1003.2,1004.5,1003.1,3.3,349.0,8.0,0.0,0.0


In [44]:
df = merge_csvs_from_list(files_to_convert)
df = df.drop_duplicates(subset=['Data', 'Hora (UTC)'])
df = df.sort_values(by=['Data', 'Hora (UTC)'])
df = df.reset_index(drop=True)
df

,Data,Hora (UTC),Temp. Ins. (C),Temp. Max. (C),Temp. Min. (C),Umi. Ins. (%),Umi. Max. (%),Umi. Min. (%),Pto Orvalho Ins. (C),Pto Orvalho Max. (C),Pto Orvalho Min. (C),Pressao Ins. (hPa),Pressao Max. (hPa),Pressao Min. (hPa),Vel. Vento (m/s),Dir. Vento (m/s),Raj. Vento (m/s),Radiacao (KJ/m²),Chuva (mm)
0,01/01/2019,0,"25,2","25,5","25,2","86,0","86,0","85,0","22,8","22,8","22,6","1008,9","1008,9","1008,5","0,7","82,0","2,8",NaN,"0,0"
1,01/01/2019,100,"25,3","25,4","25,1","86,0","87,0","86,0","22,8","22,9","22,7","1009,3","1009,4","1008,9","0,9","111,0","2,0",NaN,"0,0"
2,01/01/2019,200,"25,0","25,3","25,0","87,0","87,0","86,0","22,8","22,9","22,8","1009,1","1009,4","1009,1","1,7","100,0","2,6",NaN,"0,0"
3,01/01/2019,300,"24,7","25,1","24,7","89,0","89,0","87,0","22,7","22,9","22,6","1008,2","1009,1","1008,2","1,5","145,0","2,5",NaN,"0,0"
4,01/01/2019,400,"24,9","25,0","24,7","88,0","89,0","88,0","22,8","22,9","22,7","1008,0","1008,2","1008,0","1,3","276,0","2,0",NaN,"0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47803,31/12/2023,1900,"24,4","25,1","24,4","82,0","82,0","78,0","21,1","21,3","20,6","1011,0","1011,6","1011,0","1,3","63,0","3,5","749,00","0,0"
47804,31/12/2023,2000,"23,8","24,6","23,8","83,0","83,0","81,0","20,7","21,4","20,5","1010,8","1011,1","1010,7","4,5","95,0","6,3","470,20","0,2"
47805,31/12/2023,2100,"23,3","23,8","23,3","86,0","87,0","82,0","20,9","21,1","20,6","1011,0","1011,0","1010,8","5,1","100,0","6,5","142,10","0,0"
47806,31/12/2023,2200,"23,1","23,3","23,0","90,0","90,0","86,0","21,3","21,3","20,7","1011,7","1011,7","1011,0","4,1","82,0","6,7","20,70","0,0"


In [51]:
df.columns

Index(['DATA', 'HORA', 'Temp. Ins. (C)', 'Temp. Max. (C)', 'Temp. Min. (C)',
       'Umi. Ins. (%)', 'Umi. Max. (%)', 'Umi. Min. (%)',
       'Pto Orvalho Ins. (C)', 'Pto Orvalho Max. (C)', 'Pto Orvalho Min. (C)',
       'Pressao Ins. (hPa)', 'Pressao Max. (hPa)', 'Pressao Min. (hPa)',
       'Vel. Vento (m/s)', 'Dir. Vento (m/s)', 'Raj. Vento (m/s)',
       'Radiacao (KJ/m²)', 'Chuva (mm)'],
      dtype='object')

In [65]:
df = processar_coluna_data(df)

KeyError: 'Data'

In [49]:
df = processar_coluna_hora(df)

In [50]:
float_columns = df.columns.difference(['DATA', 'HORA'])
df = substituir_valores(df, float_columns)

In [52]:
df = df.rename(columns={
     'Temp. Ins. (C)': 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
     'Temp. Max. (C)': 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
     'Temp. Min. (C)': 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
     'Umi. Ins. (%)': 'UMIDADE RELATIVA DO AR, HORARIA (%)',
     'Umi. Max. (%)': 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
     'Umi. Min. (%)': 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
     'Pto Orvalho Ins. (C)':  'TEMPERATURA DO PONTO DE ORVALHO (°C)',
     'Pto Orvalho Max. (C)': 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
     'Pto Orvalho Min. (C)': 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
     'Pressao Ins. (hPa)': 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
     'Pressao Max. (hPa)':  'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
     'Pressao Min. (hPa)': 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
     'Vel. Vento (m/s)': 'VENTO, VELOCIDADE HORARIA (m/s)',
     'Dir. Vento (m/s)': 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
     'Raj. Vento (m/s)': 'VENTO, RAJADA MAXIMA (m/s)',
     'Radiacao (KJ/m²)': 'RADIACAO GLOBAL (KJ/m²)',
     'Chuva (mm)': 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'
})

In [53]:
df = trocar_nan_para_0(df, ['RADIACAO GLOBAL (KJ/m²)', 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)'])

In [ ]:
cols_to_interpolate = df.columns.difference(['DATA', 'HORA', 'RADIACAO GLOBAL (KJ/m²)', 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)'])
df = interpolate_slinear(df, cols_to_interpolate)

df.info()

KeyError: "['Unnamed: 19'] not found in axis"

In [ ]:
df.tail()

,Data,Hora (UTC),Temp. Ins. (C),Temp. Max. (C),Temp. Min. (C),Umi. Ins. (%),Umi. Max. (%),Umi. Min. (%),Pto Orvalho Ins. (C),Pto Orvalho Max. (C),Pto Orvalho Min. (C),Pressao Ins. (hPa),Pressao Max. (hPa),Pressao Min. (hPa),Vel. Vento (m/s),Dir. Vento (m/s),Raj. Vento (m/s),Radiacao (KJ/m²),Chuva (mm)
52027,01/01/2024,1900,"23,7","24,2","23,7","88,0","89,0","86,0","21,7","22,0","21,4","1009,8","1010,2","1009,8","1,8","97,0","3,4","398,70","0,0"
52028,01/01/2024,2000,"23,2","23,7","23,2","89,0","89,0","88,0","21,3","21,7","21,2","1009,4","1009,8","1009,4","5,4","95,0","8,0","225,80","0,0"
52029,01/01/2024,2100,"23,2","23,2","23,0","89,0","90,0","89,0","21,3","21,4","21,2","1009,8","1009,8","1009,4","5,2","93,0","8,8","190,60","0,0"
52030,01/01/2024,2200,"22,9","23,2","22,9","91,0","91,0","89,0","21,3","21,3","20,9","1010,1","1010,1","1009,8","5,4","92,0","9,6","38,40","0,0"
52031,01/01/2024,2300,"22,7","22,9","22,6","94,0","94,0","91,0","21,6","21,6","21,4","1010,7","1010,7","1010,1","2,3","73,0","7,7",NaN,"0,0"


In [ ]:
df.dtypes

DATA                                                     datetime64[ns]
HORA                                                             object
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                                float64
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)           float64
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)                 float64
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)                float64
RADIACAO GLOBAL (KJ/m²)                                         float64
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)                    float64
TEMPERATURA DO PONTO DE ORVALHO (°C)                            float64
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)                      float64
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)                      float64
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)                float64
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)                float64
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                        

In [68]:
df['DATA'] = pd.to_datetime(df['DATA'], format='%d-%m-%Y')
df['DATA'] = df['DATA'].dt.strftime('%Y-%m-%d')

In [69]:
df

,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,2019-01-01,0,25.2,25.5,25.2,86.0,86.0,85.0,22.8,22.8,22.6,1008.9,1008.9,1008.5,0.7,82.0,2.8,0.0,0.0
1,2019-01-01,100,25.3,25.4,25.1,86.0,87.0,86.0,22.8,22.9,22.7,1009.3,1009.4,1008.9,0.9,111.0,2.0,0.0,0.0
2,2019-01-01,200,25.0,25.3,25.0,87.0,87.0,86.0,22.8,22.9,22.8,1009.1,1009.4,1009.1,1.7,100.0,2.6,0.0,0.0
3,2019-01-01,300,24.7,25.1,24.7,89.0,89.0,87.0,22.7,22.9,22.6,1008.2,1009.1,1008.2,1.5,145.0,2.5,0.0,0.0
4,2019-01-01,400,24.9,25.0,24.7,88.0,89.0,88.0,22.8,22.9,22.7,1008.0,1008.2,1008.0,1.3,276.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47803,2023-12-31,19:00,24.4,25.1,24.4,82.0,82.0,78.0,21.1,21.3,20.6,1011.0,1011.6,1011.0,1.3,63.0,3.5,749.0,0.0
47804,2023-12-31,20:00,23.8,24.6,23.8,83.0,83.0,81.0,20.7,21.4,20.5,1010.8,1011.1,1010.7,4.5,95.0,6.3,470.2,0.2
47805,2023-12-31,21:00,23.3,23.8,23.3,86.0,87.0,82.0,20.9,21.1,20.6,1011.0,1011.0,1010.8,5.1,100.0,6.5,142.1,0.0
47806,2023-12-31,22:00,23.1,23.3,23.0,90.0,90.0,86.0,21.3,21.3,20.7,1011.7,1011.7,1011.0,4.1,82.0,6.7,20.7,0.0


In [54]:
df.to_csv('/content/drive/MyDrive/Trabalho Topicos IA1/new_inmet_data.csv', sep=';')

In [55]:
df_old = pd.read_csv('/content/drive/MyDrive/Trabalho Topicos IA1/gabriel_pre_test.csv', sep=';')

In [84]:
df_merge = pd.concat([df, df_old], ignore_index=True)

In [85]:
df_merge

,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",Unnamed: 0
0,2019-01-01,0,25.2,25.5,25.2,86.0,86.0,85.0,22.8,22.8,22.6,1008.9,1008.9,1008.5,0.7,82.0,2.8,0.0,0.0,NaN
1,2019-01-01,100,25.3,25.4,25.1,86.0,87.0,86.0,22.8,22.9,22.7,1009.3,1009.4,1008.9,0.9,111.0,2.0,0.0,0.0,NaN
2,2019-01-01,200,25.0,25.3,25.0,87.0,87.0,86.0,22.8,22.9,22.8,1009.1,1009.4,1009.1,1.7,100.0,2.6,0.0,0.0,NaN
3,2019-01-01,300,24.7,25.1,24.7,89.0,89.0,87.0,22.7,22.9,22.6,1008.2,1009.1,1008.2,1.5,145.0,2.5,0.0,0.0,NaN
4,2019-01-01,400,24.9,25.0,24.7,88.0,89.0,88.0,22.8,22.9,22.7,1008.0,1008.2,1008.0,1.3,276.0,2.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149707,2018-12-31,19:00,27.8,28.6,27.4,77.0,78.0,73.0,23.4,23.8,22.9,1006.7,1006.7,1006.0,6.6,130.0,8.0,1410.0,0.0,101899.0
149708,2018-12-31,20:00,26.7,27.8,26.3,82.0,83.0,77.0,23.4,23.6,22.8,1006.6,1007.1,1006.6,4.3,129.0,9.1,637.0,0.0,101900.0
149709,2018-12-31,21:00,26.1,26.8,26.1,82.0,82.0,82.0,22.8,23.4,22.8,1007.2,1007.4,1006.6,2.1,103.0,5.4,302.5,0.0,101901.0
149710,2018-12-31,22:00,25.7,26.2,25.7,84.0,84.0,82.0,22.8,22.9,22.6,1007.5,1007.5,1007.2,1.5,135.0,3.3,41.5,0.0,101902.0


In [86]:
df_merge = df_merge.drop_duplicates(subset=['DATA', 'HORA'])
df_merge = df_merge.sort_values(by=['DATA', 'HORA'])
df_merge = df_merge.reset_index(drop=True)
df_merge

,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",Unnamed: 0
0,2007-05-18,00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
1,2007-05-18,01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0
2,2007-05-18,02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,2.0
3,2007-05-18,03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,3.0
4,2007-05-18,04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149707,2024-06-14,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
149708,2024-06-14,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
149709,2024-06-14,700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
149710,2024-06-14,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [87]:
df_merge = interpolate_slinear(df_merge, ['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 'UMIDADE RELATIVA DO AR, HORARIA (%)'])

In [88]:
df_merge = df_merge.fillna(0)

In [89]:
df_merge = df_merge.drop(columns='Unnamed: 0')

In [90]:
df_merge

,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,2007-05-18,00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2007-05-18,01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007-05-18,02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2007-05-18,03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007-05-18,04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149707,2024-06-14,500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149708,2024-06-14,600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149709,2024-06-14,700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149710,2024-06-14,800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
df_merge = df_merge[df_merge['DATA'] < '2024-02-29']

In [92]:
df_merge

,DATA,HORA,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"VENTO, VELOCIDADE HORARIA (m/s)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)",RADIACAO GLOBAL (KJ/m²),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,2007-05-18,00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2007-05-18,01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007-05-18,02:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2007-05-18,03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007-05-18,04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147139,2024-02-28,500,25.7,26.3,25.6,99.0,99.0,97.0,25.5,26.0,25.3,1005.5,1006.6,1005.2,3.2,66.0,8.0,0.0,0.0
147140,2024-02-28,600,26.7,26.8,25.2,97.0,100.0,97.0,26.1,26.2,25.0,1005.1,1005.8,1005.0,1.7,352.0,6.3,0.0,0.0
147141,2024-02-28,700,26.9,26.9,26.3,95.0,97.0,95.0,26.0,26.1,25.7,1004.5,1005.2,1004.5,2.1,15.0,4.9,0.0,0.0
147142,2024-02-28,800,26.8,27.0,26.7,93.0,95.0,93.0,25.6,26.0,25.6,1003.2,1004.5,1003.1,3.3,349.0,8.0,0.0,0.0


In [19]:
df2.to_csv('/content/drive/MyDrive/Trabalho Topicos IA1/final_inmet_data.csv', sep=';')